In [1]:
%set_env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
from datasets import load_from_disk
import evaluate
import numpy as np
import torch
from transformers import VisionEncoderDecoderModel, AutoImageProcessor, AutoTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments, default_data_collator, EarlyStoppingCallback

device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    torch.backends.cuda.matmul.allow_tf32 = True
    
    n_gpus = torch.cuda.device_count()
    print("GPUs:")
    for i in range(n_gpus):
        print(torch.cuda.get_device_name(i))

GPUs:
NVIDIA RTX A6000


In [3]:
# TimeSformer
encoder = "facebook/timesformer-base-finetuned-k600"
decoder = "gpt2"

image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
tokenizer = AutoTokenizer.from_pretrained(decoder)
model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(encoder, decoder).to(device)

tokenizer.pad_token = tokenizer.eos_token
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.pad_token_id = tokenizer.pad_token_id

# VideoMAE
# encoder = "MCG-NJU/videomae-base"
# decoder = "gpt2"

# image_processor = AutoImageProcessor.from_pretrained(encoder)
# tokenizer = AutoTokenizer.from_pretrained(decoder)
# model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(encoder, decoder).to(device)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of the model checkpoint at facebook/timesformer-base-finetuned-k600 were not used when initializing TimesformerModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing TimesformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TimesformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.7.crossattention.bias', 'h.11.crossattention.c_proj.weight', 'h.8.cross

In [4]:
dataset = load_from_disk("dataset/processed/8frames_pt1")

train_idxs = [215, 224, 231, 241, 244, 254, 358, 437, 469, 595, 601, 617, 621, 631, 634, 667, 734, 739, 759, 869, 991, 997, 1028, 1062, 1081, 1083, 1094, 1106, 1108, 1113, 1127, 1131, 1136, 1141, 1155, 1162, 1220, 1263, 1267, 1278, 1285, 1299, 1330, 1331, 1342, 1344, 1346, 1347, 1372, 1381, 1391, 1398, 1440, 1491, 1778, 1837, 1857, 1863, 1898, 1904, 1913, 1923, 1925, 1928, 1946, 1953, 1962, 2016, 2043, 2047, 2050, 2119, 2181, 2232, 2235, 2239, 2240, 2245, 2246, 2253, 2254, 2261, 2262, 2264, 2343, 2345, 2353, 2359, 2362, 2378, 2382, 2390, 2391, 2396, 2406, 2408, 2414, 2415, 2416, 2417, 2418, 2426, 2428, 2431, 2438, 2439, 2440, 2442, 2443, 2444, 2446, 2449, 2450, 2451, 2454, 2470, 2571, 2619, 2623, 2718, 2720, 2763, 2768, 2825, 2852, 2927, 2933, 2958, 2968, 2972, 3010, 3110, 3155, 3280, 3441, 3457, 3458, 3474, 3483, 3493, 3515, 3524, 3525, 3576, 3638, 3647, 3662, 3711, 3713, 3741, 3742, 3743, 3744, 3745, 3746, 3747, 3748, 3749, 3750, 3751, 3753, 3754, 3755, 3756, 3757, 3758, 3759, 3760, 3761, 3762, 3763, 3764, 3766, 3767, 3768, 3769, 3771, 3772, 3773, 3775, 3776, 3777, 3778, 3779, 3780, 3781, 3782, 3847, 3848, 3863, 3875, 4065, 4111, 4158, 4302, 4323, 4326, 4332, 4337, 4338, 4345, 4349, 4352, 4358, 4360, 4362, 4364, 4368, 4369, 4376, 4388, 4398, 4399, 4417, 4422, 4424, 4428, 4433, 4434, 4435, 4436, 4438, 4440, 4442, 4444, 4446, 4447, 4448, 4450, 4451, 4452, 4453, 4454, 4456, 4457, 4458, 4460, 4462, 4463, 4464, 4513, 4520, 4523, 4526, 4529, 4546, 4556, 4559, 4560, 4566, 4711, 4932, 5012, 5022, 5029, 5039, 5049, 5164, 5200, 5215, 5223, 5237, 5246, 5255, 5270, 5470, 5514, 5515, 5516, 5518, 5520, 5521, 5525, 5527, 5528, 5529, 5533, 5536, 5542, 5544, 5545, 5547, 5625, 5672, 5678, 5765, 5843, 5937, 5944, 5950, 5953, 5968, 6037, 6057, 6240, 6279, 6323, 6342, 6348, 6410, 6412, 6465, 6474, 6488, 6538, 6539, 6541, 6542, 6550, 6553, 6557, 6560, 6562, 6564, 6565, 6566, 6567, 6569, 6571, 6572, 6573, 6578, 6594, 6605, 6611, 6615, 6684, 6722, 6732, 6757, 7039, 7146, 7170, 7345, 7358, 7369, 7373, 7471, 7520, 7521, 7522, 7523, 7525, 7527, 7528, 7531, 7533, 7534, 7535, 7536, 7538, 7539, 7541, 7542, 7543, 7550, 7552, 7615, 7667, 7709, 7725, 7727, 7732, 7737, 7739, 7742, 7747, 7756, 7766, 7776, 7813, 7848, 8026, 8051, 8054, 8073, 8100, 8182, 8195, 8200, 8206, 8216, 8229, 8240, 8266, 8289, 8340, 8347, 8350, 8358, 8362, 8366, 8367, 8370, 8378, 8400, 8406, 8409, 8411, 8416, 8423, 8439, 8456, 8463, 8487, 8543, 8561, 8568, 8596, 8614, 8718, 8767, 8777, 8871, 9058, 9296, 9356, 9362, 9561, 9581, 9586, 9594, 9600, 9607, 9657, 9794, 9808, 9816, 9831, 9861, 9864, 9905, 9908, 9947, 9948, 9955, 9961, 9964, 9965, 9971, 9980, 9981, 9982, 9983, 9990, 9991, 9996, 9997, 9998, 9999, 10000, 10001, 10003, 10004, 10006, 10007, 10008, 10009, 10010, 10015, 10017, 10194, 10212, 10217, 10222, 10227, 10230, 10231, 10233, 10237, 10241, 10242, 10244, 10269, 10282, 10283, 10285, 10288, 10291, 10293, 10297, 10301, 10304, 10306, 10384, 10389, 10419, 10484, 10507, 10577, 10599, 10654, 10663, 10666, 10670, 10673, 10675, 10691, 10706, 10728, 11043, 11048, 11052, 11055, 11063, 11193, 11198, 11203, 11206, 11207, 11209, 11212, 11218, 11230, 11288, 11349, 11362, 11373, 11408, 11423, 11484, 11649, 11664, 11712, 11841, 11845, 11858, 11860, 11865, 11866, 11896, 11926, 11928, 11930, 11931, 11935, 11942, 11953, 11954, 11956, 11968, 11989, 12005, 12010, 12012, 12013, 12017, 12018, 12025, 12027, 12030, 12033, 12035, 12038, 12050, 12051, 12062, 12078, 12096, 12114, 12118, 12121, 12124, 12130, 12133, 12134, 12144, 12145, 12150, 12154, 12157, 12159, 12160, 12161, 12163, 12166, 12172, 12175, 12180, 12183, 12187, 12197, 12200, 12203, 12207, 12208, 12221, 12245, 12255, 12256, 12260, 12264, 12270, 12275, 12277, 12280, 12283, 12286, 12289, 12302, 12306, 12327, 12330, 12335, 12339, 12343, 12355, 12361, 12370, 12378, 12397, 12402, 12408, 12465, 12736, 12806, 12915, 13122, 13178, 13189, 13224, 13314, 13319, 13321, 13497, 13514, 13517, 13579, 13600, 13606, 13615, 13616, 13623, 13628, 13630, 13633, 13635, 13637, 13664, 13665, 13666, 13667, 13668, 13669, 13671, 13672, 13673, 13674, 13675, 13676, 13677, 13679, 13680, 13681, 13682, 13683, 13684, 13685, 13686, 13687, 13688, 13689, 13690, 13691, 13692, 13694, 13695, 13696, 13697, 13698, 13699, 13700, 13701, 13702, 13703, 13738, 13746, 13758, 13759, 13880, 13912, 13919, 13946, 13950, 14070, 14147, 14154, 14155, 14157, 14165, 14172, 14199, 14264, 14405, 14436, 14448, 14453, 14455, 14458, 14459, 14465, 14467, 14468, 14469, 14473, 14474, 14475, 14479, 14480, 14481, 14483, 14486, 14496, 14519, 14566, 14594, 14634, 14684, 14772, 14787, 14791, 14792, 14796, 14799, 14802, 14803, 14807, 14809, 14818, 14822, 14828, 14830, 14834, 14836, 14839, 14843, 14845, 14849, 14877, 14878, 14881, 14925, 14971, 14977, 14998, 15009, 15054, 15186, 15192, 15193, 15201, 15202, 15203, 15212, 15216, 15217, 15220, 15221, 15224, 15258, 15274, 15277, 15291, 15294, 15305, 15335, 15358, 15414, 15479, 15567, 15645, 15660, 15664, 15671, 15672, 15675, 15680, 15683, 15686, 15759, 15896, 15897, 15899, 15901, 15971, 15989, 16043, 16073, 16159, 16296, 16297, 16299, 16315, 16337, 16355, 16400, 16411, 16417, 16419, 16443, 16565, 16566, 16567, 16572, 16573, 16574, 16579, 16581, 16584, 16585, 16586, 16587, 16589, 16590, 16593, 16594, 16596, 16598, 16604, 16605, 16611, 16617, 16618, 16619, 16620, 16622, 16624, 16628, 16629, 16630, 16633, 16634, 16636, 16637, 16638, 16648, 16660, 16662, 16672, 16680, 16685, 16697, 16703, 16707, 16710, 16713, 16722, 16728, 16745, 16756, 16793, 16799, 16800, 16801, 16803, 16804, 16805, 16809, 16811, 16812, 16813, 16816, 16818, 16819, 16822, 16823, 16824, 16825, 16827, 16828, 16829, 16831, 16833, 16838, 16840, 16842, 16843, 16844, 16845, 16847, 16848, 16852, 16853, 16863, 16865, 16867, 16868, 16923, 16995, 17123, 17133, 17145, 17146, 17149, 17166, 17189, 17192, 17221, 17287, 17311, 17336, 17449, 17509, 17511, 17513, 17517, 17518, 17519, 17520, 17521, 17524, 17525, 17526, 17527, 17528, 17529, 17532, 17540, 17541, 17542, 17543, 17545, 17592, 17600, 17625, 17662, 17678, 17686, 17734, 17754, 17763, 17778, 17996, 17997, 17998, 17999, 18000, 18001, 18002, 18004, 18009, 18010, 18013, 18014, 18015, 18017, 18018, 18019, 18020, 18023, 18025, 18029, 18030, 18031, 18034, 18036, 18037, 18038, 18041, 18099, 18167, 18177, 18186, 18248, 18249, 18256, 18264, 18271, 18434, 18597, 18598, 18601, 18608, 18613, 18622, 18634, 18654, 18735, 18741, 18807, 18835, 18840, 18841, 18862, 18887, 18966, 18973, 18974, 18986, 19197, 19274, 19384, 19436, 19438, 19449, 19454, 19472, 19474, 19480, 19487, 19490, 19491, 19492, 19493, 19496, 19497, 19502, 19505, 19506, 19508, 19520, 19521, 19523, 19592, 19642, 19728, 19794, 19797, 19815, 19818, 19852, 19867, 19905, 19967, 20099, 20100, 20101, 20102, 20103, 20106, 20109, 20110, 20112, 20115, 20116, 20117, 20118, 20119, 20123, 20124, 20125, 20126, 20127, 20130, 20131, 20132, 20133, 20136, 20137, 20144, 20227, 20265, 20276, 20310, 20314, 20472, 20509, 20571, 20572, 20573, 20575, 20576, 20580, 20581, 20582, 20583, 20585, 20587, 20588, 20590, 20591, 20593, 20594, 20595, 20596, 20597, 20600, 20601, 20603, 20604, 20605, 20606, 20608, 20609, 20612, 20661, 20664, 20694, 20695, 20698, 20699, 20701, 20703, 20711, 20713, 20715, 20716, 20720, 20721, 20727, 20751, 20756, 20766, 20767, 20768, 20769, 20770, 20772, 20775, 20778, 20779, 20781, 20782, 20784, 20786, 20787, 20788, 20789, 20793, 20794, 20799, 20802, 20805, 20807, 20810, 20838, 20861, 20862, 20871, 20890, 20892, 20893, 21067, 21073, 21114, 21115, 21120, 21327, 21341, 21416, 21422, 21467, 21561, 21600, 21608, 21611, 21617, 21629, 21634, 21635, 21637, 21641, 21662, 21901, 21904, 21910, 21916, 21921, 21924, 22132, 22135, 22139, 22170, 22174, 22178, 22181, 22195, 22353, 22359, 22385, 22386, 22387, 22391, 22429, 22432, 22433, 22442, 22448, 22450, 22451, 22459, 22462, 22463, 22464, 22469, 22470, 22474, 22486, 22489, 22490, 22495, 22501, 22504, 22889]
val_idxs = [6, 40, 41, 66, 67, 105, 118, 136, 185, 188, 193, 194, 195, 196, 197, 230, 252, 259, 262, 263, 271, 332, 338, 354, 357, 358, 393, 444, 463, 496, 519, 566, 589, 611, 631, 633, 654, 707, 709, 714, 731, 781, 782, 908, 926, 946, 947, 948, 949, 950, 1010, 1012, 1013, 1014, 1015, 1028, 1031, 1066, 1069, 1078, 1080, 1087, 1088, 1091, 1103, 1104, 1145, 1171, 1173, 1226, 1230, 1231, 1250, 1304, 1348, 1399, 1433, 1440, 1483, 1525, 1526, 1529, 1657, 1674, 1675, 1727, 1736, 1737, 1762, 1802, 1882, 1926, 1927, 1930, 1931, 1932, 1933, 1934, 1994, 2021, 2032, 2081, 2082, 2085, 2086, 2098, 2100, 2102, 2107, 2120, 2123, 2125, 2129, 2180, 2193, 2196, 2198, 2262, 2316, 2317, 2341, 2347, 2507, 2512, 2533, 2536, 2571, 2590, 2592, 2594, 2597, 2600, 2609, 2618, 2632, 2635, 2637]

dataset["train"] = dataset["train"].select(train_idxs)
dataset["validation"] = dataset["validation"].select(val_idxs)
dataset

DatasetDict({
    train: Dataset({
        features: ['pixel_values', 'labels'],
        num_rows: 100
    })
    validation: Dataset({
        features: ['pixel_values', 'labels'],
        num_rows: 10
    })
})

In [5]:
training_args = Seq2SeqTrainingArguments(
    output_dir="training/test",
    tf32=True,
    predict_with_generate=True,
    load_best_model_at_end=True,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    dataloader_num_workers=4,
    num_train_epochs=5,
    learning_rate=1,
)

metric = evaluate.load("rouge")
def metrics(eval_preds):
    preds, labels = eval_preds
    preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return metric.compute(predictions=preds, references=labels)

trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=image_processor,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=default_data_collator,
    compute_metrics=metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)],
)

trainer.train()

/home/922201615/anaconda3/envs/vlm/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 100
  Num Epochs = 5
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 125
  Number of trainable parameters = 274065408


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,4148.697800,10936.162109,0.000000,0.000000,0.000000,0.000000
2,12651.220000,nan,0.000000,0.000000,0.000000,0.000000
3,0.000000,nan,0.000000,0.000000,0.000000,0.000000


***** Running Evaluation *****
  Num examples = 10
  Batch size = 4
Generate config GenerationConfig {
  "decoder_start_token_id": 50256,
  "pad_token_id": 50256,
  "transformers_version": "4.26.1"
}

/home/922201615/anaconda3/envs/vlm/lib/python3.10/site-packages/transformers/generation/utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Generate config GenerationConfig {
  "decoder_start_token_id": 50256,
  "pad_token_id": 50256,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 50256,
  "pad_token_id": 50256,
  "transformers_version": "4.26.1"
}

Saving model checkpoint to training/test/checkpoint-25
Configuration saved in training/test/checkpoint-25/

TrainOutput(global_step=75, training_loss=5599.972604166666, metrics={'train_runtime': 138.7853, 'train_samples_per_second': 3.603, 'train_steps_per_second': 0.901, 'total_flos': 5.0869658714112e+17, 'train_loss': 5599.972604166666, 'epoch': 3.0})